## Introduction

Hello, my readers, I'm Parthiv. Quick note: **This file will have a lot of playing around and searching for trends!** The `dist/` directory will have a lot of the final graphs and analysis, so make sure to explore that after. I'll try to explain what I'm doing as we go on, but I apologize if anything's unclear.

In [3]:
# Importing necessary libraries
# pip install pandas numpy matplotlib seaborn plotly scienceplots

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px


In [5]:
# Config for Matplotlib, Seaborn, Plotly, LaTeX

# Set global figure size
plt.rcParams['figure.figsize'] = [10, 6]

# Set font sizes
plt.rcParams['font.size'] = 12
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['axes.titlesize'] = 16

plt.rcParams['figure.autolayout'] = True

sns.set_style('whitegrid')
sns.set_context('notebook', font_scale=1.2)

import plotly.io as pio
pio.templates.default = "plotly_white"

plt.rcParams['text.usetex'] = True
